## The below code is currently needed to make finitediff install on Kaggle:

In [ ]:
mypaths = !python -c "import sysconfig; print(sysconfig.get_paths()['purelib'])"
mypath = mypaths[0]
!pip install trash-cli
!trash $mypath/numpy*
!pip install --upgrade numpy
!pip install finitediff
import os
os.kill(os.getpid(), 9)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for trash-cli: filename=trash_cli-0.23.2.13.2-py3-none-any.whl size=70574 sha256=6c3786f6785bea2653a507ef9ce487075aac404dd3202e5cfc141ab3fef46bc4
  Stored in directory: /root/.cache/pip/wheels/81/0f/15/47be3464f17641d8b18427ac54fb89d2360d4d2faa809e2809
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26324 sha256=f207d37063f21d739c7b53d305c2f522ef6f9322ae9be5b159c9514dbb95174f
  Stored in directory: /root/.cache/pip/wheels/7c/d0/9e/1f26ebb66d9e1732e4098bc5a6c2d91f6c9a529838f0284890
Successfully built trash-cli typing
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 52.0 MB/s eta 0:00:0000:0100:01
ERROR: pi

## After this step install HyDesign as usual:

In [1]:
!pip install git+https://gitlab.windenergy.dtu.dk/TOPFARM/hydesign.git

  Cloning https://gitlab.windenergy.dtu.dk/TOPFARM/hydesign.git to /tmp/pip-req-build-xvf8jqqb
  Running command git clone --filter=blob:none --quiet https://gitlab.windenergy.dtu.dk/TOPFARM/hydesign.git /tmp/pip-req-build-xvf8jqqb
done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 MB 25.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 633.5/633.5 kB 37.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.5/29.5 MB 5.1 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.5/206.5 kB 17.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 70.2 MB/s eta 0:00:00:00:0100:01
  Created wheel for hydesign: filename=hydesign-1.1.2-py3-none-any.whl size=17615665 sha256=fcb1a271bab2cbc11060

## Import HyDesign:

In [2]:
from hydesign.hpp_assembly_P2X import hpp_model_P2X as hpp_model
from hydesign.Parallel_EGO import get_kwargs, EfficientGlobalOptimizationDriver
from smt.applications.mixed_integer import FLOAT, INT

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Detect no of CPUs to setup optimization alogrithm

In [3]:
import multiprocessing
n_cpu = int(multiprocessing.cpu_count())
n_doe = int(n_cpu * 4)
n_clusters = int(n_cpu / 2)

In [4]:
inputs = {
    'example': 4,
    'name': None,
    'longitude': None,
    'latitude': None,
    'altitude': None,
    'input_ts_fn': None,
    'sim_pars_fn': None,

    'opt_var': "NPV_over_CAPEX",
    'num_batteries': 1,
    'n_procs': n_cpu,
    'n_doe': n_doe,
    'n_clusters': n_clusters,
    'n_seed': 0,
    'max_iter': 10,
    'final_design_fn': 'hydesign_design_0.csv',
    'npred': 3e4,
    'tol': 1e-6,
    'min_conv_iter': 2,
    'work_dir': './',
    'hpp_model': hpp_model,
    }

kwargs = get_kwargs(inputs)
kwargs['variables'] = {
    'clearance [m]':
        {'var_type':'design',
          'limits':[10, 60],
          'types':INT
          },
        # {'var_type':'fixed',
        #   'value': 35
        #   a},
     'sp [W/m2]':
        {'var_type':'design',
         'limits':[200, 360],
         'types':INT
         },
    'p_rated [MW]':
        {'var_type':'design',
          'limits':[1, 10],
          'types':INT
          },
        # {'var_type':'fixed',
        #  'value': 6
         # },
    'Nwt':
        {'var_type':'design',
          'limits':[0, 400],
          'types':INT
          },
        # {'var_type':'fixed',
        #   'value': 200
        #   },
    'wind_MW_per_km2 [MW/km2]':
        # {'var_type':'design',
        #   'limits':[5, 9],
        #   'types':FLOAT
        #   },
        {'var_type':'fixed',
          'value': 7
          },
    'solar_MW [MW]':
        # {'var_type':'design',
        #   'limits':[0, 400],
        #   'types':INT
        #   },
        {'var_type':'fixed',
          'value': 200
          },
    'surface_tilt [deg]':
        # {'var_type':'design',
        #   'limits':[0, 50],
        #   'types':FLOAT
        #   },
        {'var_type':'fixed',
          'value': 25
          },
    'surface_azimuth [deg]':
        # {'var_type':'design',
        #   'limits':[150, 210],
        #   'types':FLOAT
        #   },
        {'var_type':'fixed',
          'value': 180
          },
    'DC_AC_ratio':
        # {'var_type':'design',
        #   'limits':[1, 2.0],
        #   'types':FLOAT
        #   },
        {'var_type':'fixed',
          'value':1.0,
          },
    'b_P [MW]':
        # {'var_type':'design',
        #   'limits':[0, 100],
        #   'types':INT
        #   },
        {'var_type':'fixed',
          'value': 50
          },
    'b_E_h [h]':
        # {'var_type':'design',
        #   'limits':[1, 10],
        #   'types':INT
        #   },
        {'var_type':'fixed',
          'value': 6
          },
    'cost_of_battery_P_fluct_in_peak_price_ratio':
        # {'var_type':'design',
        #   'limits':[0, 20],
        #   'types':FLOAT
        #   },
        {'var_type':'fixed',
          'value': 10},
    'ptg_MW [MW]':
        {'var_type':'design',
          'limits':[1, 50],
          'types':INT
          },

    }

EGOD = EfficientGlobalOptimizationDriver(**kwargs)

Selected example site:
---------------------------------------------------
case                                         Europe
name                               France_good_wind
longitude                                 -0.864258
latitude                                  48.744116
altitude                                      302.0
input_ts_fn    Europe/input_ts_France_good_wind.csv
sim_pars_fn                     Europe/hpp_pars.yml
price_fn                   Europe/2030-EL_PRICE.csv
price_col                                      FR_R
Name: 4, dtype: object


In [5]:
EGOD.run()





Sizing a HPP plant at France_good_wind:

longitude = -0.864258
latitude = 48.744116
altitude = 302.0



Initial 16 simulations took 2.75 minutes
  Current solution -NPV_over_CAPEX = -1.262E-01

Update sm and extract candidate points took 0.01 minutes
Check-optimal candidates: new 12 simulations took 2.02 minutes
  Current solution -NPV_over_CAPEX = -1.461E-01
  rel_yopt_change = -1.36E-01
Iteration 1 took 2.04 minutes

Update sm and extract candidate points took 0.02 minutes
Check-optimal candidates: new 12 simulations took 2.04 minutes
  Current solution -NPV_over_CAPEX = -1.635E-01
  rel_yopt_change = -1.07E-01
Iteration 2 took 2.07 minutes

Update sm and extract candidate points took 0.02 minutes
Check-optimal candidates: new 11 simulations took 1.97 minutes
  Current solution -NPV_over_CAPEX = -1.732E-01
  rel_yopt_change = -5.61E-02
Iteration 3 took 2.0 minutes

Update sm and extract candidate points took 0.02 minutes
Check-optimal candidates: new 12 simulations took 2.05 minu